# Simulate Queries and Responses from input text

## Objective

Use the Simulator to generate high-quality queries and responses from your data using LLMs.

This tutorial uses the following Azure AI services:

- Access to Azure OpenAI Service - you can apply for access [here](https://go.microsoft.com/fwlink/?linkid=2222006)
- An Azure AI Studio project - go to [aka.ms/azureaistudio](https://aka.ms/azureaistudio) to create a project

## Time

You should expect to spend 5-10 minutes running this sample. 

## About this example

Large Language Models (LLMs) can help you create query and response datasets from your existing data sources such as text or index. These datasets can be useful for various tasks, such as testing your retrieval capabilities, evaluating and improving your RAG workflows, tuning your prompts and more. In this sample, we will explore how to use the Simulator to generate high-quality query and response pairs from your data using LLMs and simulate interactions with your application with them.




### Data

In this sample we will generate text data from Wikipedia. You can follow the same steps replacing the text with any other source documents of your application's interest. Make sure that the length of the text is within the selected Azure AI model's context length.

## Before you begin



### Installation

Install the following packages required to execute this notebook. 



In [ ]:
# Install the packages
%pip install azure-identity azure-ai-evaluation
%pip install wikipedia openai

### Parameters

Lets initialize some variables. For `subscription_id`, `resource_group_name` and `project_name`, you can go to the Project Overview page in the AI Studio. Replace the items in <> with values for your project. 

In [ ]:
# project details
subscription_id: str = "<your-subscription-id>"
resource_group_name: str = "<your-resource-group>"
project_name: str = "<your-project-name>"

should_cleanup: bool = False

## Connect to your project

To start with let us create a config file with your project details. Replace the items in <> with values for your project

In [ ]:
import json
import os

azure_ai_project = {
    "subscription_id": subscription_id,
    "resource_group_name": resource_group_name,
    "project_name": project_name,
}

os.environ["AZURE_OPENAI_API_KEY"] = "<your-api-key>"
os.environ["AZURE_OPENAI_ENDPOINT"] = "<your-endpoint>"
# JSON mode supported model preferred to avoid errors ex. gpt-4o-mini, gpt-4o, gpt-4 (1106)
os.environ["AZURE_DEPLOYMENT"] = "<your-deployment>"
os.environ["AZURE_API_VERSION"] = "<api version>"

Let us connect to the project

In [ ]:
from azure.ai.evaluation.simulator import Simulator
from azure.identity import DefaultAzureCredential

simulator = Simulator(azure_ai_project=azure_ai_project, credential=DefaultAzureCredential())

Connecting the simulator to your application

In [ ]:
from typing import List, Dict, Any, Optional
from openai import AzureOpenAI


def call_to_your_ai_application(query: str) -> str:
    # logic to call your application
    # use a try except block to catch any errors
    deployment = os.environ.get("AZURE_DEPLOYMENT")
    endpoint = os.environ.get("AZURE_ENDPOINT")
    client = AzureOpenAI(
        azure_endpoint=endpoint,
        api_version=os.environ.get("AZURE_API_VERSION"),
        api_key=os.environ.get("AZURE_API_KEY"),
    )
    completion = client.chat.completions.create(
        model=deployment,
        messages=[
            {
                "role": "user",
                "content": query,
            }
        ],
        max_tokens=800,
        temperature=0.7,
        top_p=0.95,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None,
        stream=False,
    )
    message = completion.to_dict()["choices"][0]["message"]
    # change this to return the response from your application
    return message["content"]


async def callback(
    messages: List[Dict],
    stream: bool = False,
    session_state: Any = None,  # noqa: ANN401
    context: Optional[Dict[str, Any]] = None,
) -> dict:
    messages_list = messages["messages"]
    # get last message
    latest_message = messages_list[-1]
    query = latest_message["content"]
    context = None
    # call your endpoint or ai application here
    response = call_to_your_ai_application(query)
    # we are formatting the response to follow the openAI chat protocol format
    formatted_response = {
        "content": response,
        "role": "assistant",
        "context": {
            "citations": None,
        },
    }
    messages["messages"].append(formatted_response)
    return {"messages": messages["messages"], "stream": stream, "session_state": session_state, "context": context}

### Generate Query Responses from raw text
In this example we use a wikipedia article as raw text generate Query Response pairs.

In [ ]:
import wikipedia

wiki_search_term = "Leonardo da vinci"
wiki_title = wikipedia.search(wiki_search_term)[0]
wiki_page = wikipedia.page(wiki_title)
text = wiki_page.summary[:5000]

### Call to simulator
This call to the simulator generates 4 query response pairs in its first pass.
In the second pass, it picks up one task, pairs it with a query (generated in previous pass) and sends it to the configured llm to build the first user turn. This user turn is then passed to the `callback` method. The conversation continutes till the `max_conversation_turns` turns.

The output of the simulator will have the original task, original query, the original query and the response generated from the first turn as expected response. You can find them in the `context` key of the conversation.

In [ ]:
outputs = await simulator(
    target=callback,
    text=text,
    num_queries=4,
    max_conversation_turns=3,
    tasks=[
        f"I am a student and I want to learn more about {wiki_search_term}",
        f"I am a teacher and I want to teach my students about {wiki_search_term}",
        f"I am a researcher and I want to do a detailed research on {wiki_search_term}",
        f"I am a statistician and I want to do a detailed table of factual data concerning {wiki_search_term}",
    ],
)

### Save the generated data for later use

In [ ]:
from pathlib import Path

output_file = Path("output.json")
with output_file.open("a") as f:
    json.dump(outputs, f)

## Cleaning up

To clean up all Azure ML resources used in this example, you can delete the individual resources you created in this tutorial.

If you made a resource group specifically to run this example, you could instead [delete the resource group](https://learn.microsoft.com/en-us/azure/azure-resource-manager/management/delete-resource-group).